## Deliverable 2. Create a Customer Travel Destinations Map.

In [19]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
# From https://stackoverflow.com/questions/59228094/module-not-found-error-no-module-named-config
from config import g_key



# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [20]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barguzin,RU,53.6167,109.6167,36.75,54,93,8.39,overcast clouds
1,1,Erdenet,MN,49.0333,104.0833,35.89,35,100,12.53,overcast clouds
2,2,Busselton,AU,-33.6500,115.3333,72.00,54,4,1.99,clear sky
3,3,Lebu,CL,-37.6167,-73.6500,54.19,85,8,4.54,clear sky
4,4,Cockburn Town,TC,21.4612,-71.1419,73.62,63,28,10.83,scattered clouds


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Busselton,AU,-33.6500,115.3333,72.00,54,4,1.99,clear sky
4,4,Cockburn Town,TC,21.4612,-71.1419,73.62,63,28,10.83,scattered clouds
7,7,Sao Joao Da Barra,BR,-21.6403,-41.0511,70.03,79,92,8.55,overcast clouds
12,12,Road Town,VG,18.4167,-64.6167,75.00,76,4,3.00,clear sky
14,14,Avarua,CK,-21.2078,-159.7750,71.60,94,40,2.30,scattered clouds
18,18,Vao,NC,-22.6667,167.4833,75.33,78,100,19.15,overcast clouds
19,19,Rikitea,PF,-23.1203,-134.9692,78.12,74,4,9.82,light rain
30,30,Butaritari,KI,3.0707,172.7902,81.32,78,99,8.23,moderate rain
31,31,Taoudenni,ML,22.6783,-3.9836,80.35,25,0,17.56,clear sky
35,35,Mabaruma,GY,8.2000,-59.7833,72.34,94,94,2.08,overcast clouds


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             158
City                   158
Country                158
Lat                    158
Lng                    158
Max Temp               158
Humidity               158
Cloudiness             158
Wind Speed             158
Current Description    158
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Busselton,AU,72.00,clear sky,-33.6500,115.3333,
4,Cockburn Town,TC,73.62,scattered clouds,21.4612,-71.1419,
7,Sao Joao Da Barra,BR,70.03,overcast clouds,-21.6403,-41.0511,
12,Road Town,VG,75.00,clear sky,18.4167,-64.6167,
14,Avarua,CK,71.60,scattered clouds,-21.2078,-159.7750,
18,Vao,NC,75.33,overcast clouds,-22.6667,167.4833,
19,Rikitea,PF,78.12,light rain,-23.1203,-134.9692,
30,Butaritari,KI,81.32,moderate rain,3.0707,172.7902,
31,Taoudenni,ML,80.35,clear sky,22.6783,-3.9836,
35,Mabaruma,GY,72.34,overcast clouds,8.2000,-59.7833,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [43]:
# clean_df = hotel_df.dropna()
# https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas
    
import numpy as np
clean_df = hotel_df.replace(r'', np.nan, regex=True)
clean_hotel_df = clean_df.dropna()
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Busselton,AU,72.00,clear sky,-33.6500,115.3333,Observatory Guest House
4,Cockburn Town,TC,73.62,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
7,Sao Joao Da Barra,BR,70.03,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas
12,Road Town,VG,75.00,clear sky,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
14,Avarua,CK,71.60,scattered clouds,-21.2078,-159.7750,Paradise Inn
18,Vao,NC,75.33,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny
19,Rikitea,PF,78.12,light rain,-23.1203,-134.9692,Pension Maro'i
30,Butaritari,KI,81.32,moderate rain,3.0707,172.7902,Isles Sunset Lodge
35,Mabaruma,GY,72.34,overcast clouds,8.2000,-59.7833,Monty Brooms Guest House
39,Port Alfred,ZA,72.70,scattered clouds,-33.5906,26.8910,The Halyards Hotel


In [45]:
# 8a. Create the output File (CSV)
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")
# 8b. Export the City_Data into a csv
#clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [62]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dt>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [63]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))